In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext

#sc = SparkContext()

charts_df = spark.table("weekly_charts_table")
charts_df.show(3)
charts_df.printSchema()

+-------+----------+--------------------+
country| day| songs|
+-------+----------+--------------------+
 hk|2019-05-23|[[st -> 141671, i...|
 hk|2019-05-30|[[st -> 121714, i...|
 hk|2019-06-06|[[st -> 115802, i...|
+-------+----------+--------------------+
only showing top 3 rows

root
-- country: string (nullable = true)
-- day: string (nullable = true)
-- songs: array (nullable = true)
 |-- element: map (containsNull = true)
 | |-- key: string
 | |-- value: string (valueContainsNull = true)

In [2]:
#UDF to get the ids separated with space as a document
from pyspark.sql.functions import udf

@udf(T.ArrayType(T.StringType()))
def get_ids(songs):
  return [s["id"] for s in songs]

charts_df = charts_df.withColumn("songs_array", get_ids(charts_df.songs))

In [3]:
charts_df.count()

Out[52]: 10603

In [4]:
charts_time_df = charts_df.filter("day = '2019-05-23'")
charts_time_df.count()

Out[53]: 62

In [5]:
#Count number of songs in each country
charts_time_df.select("country", F.size(charts_time_df.songs_array).alias('n')).show()

+-------+---+
country| n|
+-------+---+
 hk|200|
 id|200|
 be|200|
 it|200|
 vn|200|
 pt|200|
 jp|200|
 za|200|
 is|200|
 ec|200|
 lu|200|
 mx|200|
 do|200|
 es|200|
 ee|200|
 pe|200|
 nz|200|
 se|200|
 ni|200|
 ro|200|
+-------+---+
only showing top 20 rows

In [6]:
charts_time_df.limit(1).select("songs_array").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="songs_array", outputCol="features").fit(charts_time_df)
cv_result = cv.transform(charts_time_df)
cv_result.show()

+-------+----------+--------------------+--------------------+--------------------+
country| day| songs| songs_array| features|
+-------+----------+--------------------+--------------------+--------------------+
 hk|2019-05-23|[[st -> 141671, i...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|
 id|2019-05-23|[[st -> 1892259, ...|[1iWHabUgUuuKLBa0...|(3854,[0,1,2,3,4,...|
 be|2019-05-23|[[st -> 381505, i...|[6u7jPi22kF8CTQ3r...|(3854,[0,1,2,3,4,...|
 it|2019-05-23|[[st -> 2889826, ...|[45zJRuzK9e6AO527...|(3854,[0,1,2,3,4,...|
 vn|2019-05-23|[[st -> 105559, i...|[5KawlOMHjWeUjQtn...|(3854,[0,1,2,3,4,...|
 pt|2019-05-23|[[st -> 259153, i...|[1ai9FDnFQDv18WAt...|(3854,[0,1,2,3,4,...|
 jp|2019-05-23|[[st -> 459664, i...|[2cKiHG16JBiqHdi7...|(3854,[0,1,2,3,4,...|
 za|2019-05-23|[[st -> 65710, id...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|
 is|2019-05-23|[[st -> 94264, id...|[4szQkW1CDHqbRZGl...|(3854,[0,1,2,3,4,...|
 ec|2019-05-23|[[st -> 492129, i...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|
 lu|2019-05-23|[[st -> 23009, id...|[2kkcJ8tSDV42LdWc...|(3854,[0,1,2,3,4,...|
 mx|2019-05-23|[[st -> 5201633, ...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|
 do|2019-05-23|[[st -> 168992, i...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|
 es|2019-05-23|[[st -> 3286813, ...|[1c0hsvHLELX6y8qy...|(3854,[0,1,2,3,4,...|
 ee|2019-05-23|[[st -> 60214, id...|[1Xi84slp6FryDSCb...|(3854,[0,1,2,3,4,...|
 pe|2019-05-23|[[st -> 1233597, ...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,3,4,5,...|
 nz|2019-05-23|[[st -> 472495, i...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|
 se|2019-05-23|[[st -> 2086130, ...|[2kju4JyRnNmiwQYh...|(3854,[0,1,2,3,4,...|
 ni|2019-05-23|[[st -> 43210, id...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|
 ro|2019-05-23|[[st -> 65089, id...|[6u7jPi22kF8CTQ3r...|(3854,[0,1,2,3,4,...|
+-------+----------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [8]:
cv.vocabulary

Out[57]: ['2Fxmhks0bxGSBdJ92vM42m',
 '6ocbgoVGwYJhOv1GgI9NsF',
 '6u7jPi22kF8CTQ3rb9DHE7',
 '3HVWdVOQ0ZA45FuZGSfvns',
 '4y3OI86AEP6PQoDE6olYhO',
 '2dpaYNEQHiRxtZbfNsse99',
 '43zdsphuZLzwA9k4DJhU0I',
 '6LsAAHotRLMOHfCsSfYCsz',
 '2VxeLyX666F8uXCJ0dZF8B',
 '4SSnFejRGlZikf02HLewEF',
 '4MsUsFfWseB9M2plGJyuIw',
 '3KkXRkHbMCARz0aVfEt68P',
 '6Qs4SXO9dwPj5GKvVOv8Ki',
 '53CJANUxooaqGOtdsBTh7O',
 '5KawlOMHjWeUjQtnuRs22c',
 '5p7ujcrUXASCNwRaWNHR1C',
 '7qiZfU4dY1lWllzX7mPBI3',
 '4u7EnebtmKWzUH433cf5Qv',
 '340UVheS8z3ncW9TTUhAbc',
 '3e9HZxeyfWwjeyPAMmWSSQ',
 '3Fj47GNK2kUF0uaEDgXLaD',
 '0u2P5u6lvoDfwTYjAADbn4',
 '6nDKrPlXdpomGBgAlO7UdP',
 '7fa9MBXhVfQ8P8Df9OEbD8',
 '5w9c2J52mkdntKOmRLeM2m',
 '6MWtB6iiXyIwun0YzU6DFP',
 '25sgk305KZfyuqVBQIahim',
 '6TqXcAFInzjp0bODyvrWEq',
 '7qEHsqek33rTcFNT9PFqLf',
 '5hVghJ4KaYES3BFUATCYn0',
 '4pUCKHjJ4Ijewc37rRrvHn',
 '3y3brCCecHC3Db18aIOnny',
 '5icOoE6VgqFKohjWWNp0Ac',
 '7Feaw9WAEREY0DUOSXJLOM',
 '4w8niZpiMy6qz1mntFA5uM',
 '0mjAU3yKR1QnXnHtjGJqTM',
 '7COXchtUOMd6uIT6HvmRaI',
 '7njKgVl3AnLflRad38o7AG',
 '2qxmye6gAegTMjLKEBoR3d',
 '0uUNN1nSoUx1A4fkDCWDQ7',
 '0e7ipj03S05BNilyu5bRzt',
 '5WHTFyqSii0lmT9R21abT8',
 '5qmq61DAAOUaW8AUo8xKhh',
 '2MuMD298IpYqKF367wegUd',
 '1rqqCSm0Qe4I9rUvWncaom',
 '0tgVpDi06FyKpA1z0VMD4v',
 '0E9ZjEAyAwOXZ7wJC0PD33',
 '4kV4N9D1iKVxx1KLvtTpjS',
 '0pqnGHJpmpxLKifKRmU6WP',
 '7FGq80cy8juXBCD2nrqdWU',
 '5JEx7HbmvHQQswJCsoo9rA',
 '0BnTBAGmr9FtYwkZrwKhwS',
 '4n7jnSxVLd8QioibtTDBDq',
 '5qrSlOut2rNAWv3ubArkNy',
 '7dt6x5M1jzdTEt8oCbisTK',
 '4PkIDTPGedm0enzdvilLNd',
 '2BgEsaKNfHUdlh97KmvFyo',
 '1DFD5Fotzgn6yYXkYsKiGs',
 '2iUXsYOEPhVqEBwsqP70rE',
 '5N5k9nd479b1xpDZ4usjrg',
 '1tNJrcVe6gwLEiZCtprs1u',
 '1WSGTYKLEpKfNQYYW5Oymt',
 '1LIqNVhaIgxXUt2eVoIQJ3',
 '5iwz1NiezX7WWjnCgY5TH4',
 '3XF5xLJHOQQRbWya6hBp7d',
 '4f8Mh5wuWHOsfXtzjrJB3t',
 '0b6UoVjUzFyZNpHI0aAhKt',
 '5QiNbIilmMiMBXvaAlf7lN',
 '2gwkD6igEhQbDQegRCcdoB',
 '2S1LebN6AXXQqJolBxlWgO',
 '1wJRveJZLSb1rjhnUHQiv6',
 '6jfEZdUJ2Yla6h7gTJgKuo',
 '4szQkW1CDHqbRZGltXJdd3',
 '4XZxalvnG4Yo8YZLuiT1kV',
 '1c0hsvHLELX6y8qymnpLKL',
 '51RN0kzWd7xeR4th5HsEtW',
 '1Xi84slp6FryDSCbzq4UCD',
 '5QvBXUm5MglLJ3iBfTX2Wo',
 '4cbdPT6uaBOgOQe3fLMofl',
 '5itOtNx0WxtJmi1TQ3RuRd',
 '116H0KvKr2Zl4RPuVBruDO',
 '79esEXlqqmq0GPz0xQSZTV',
 '4NSW0Km5ZG60L8FthUebPJ',
 '6bOkaEXc5CopinGazSLokx',
 '6IRdLKIyS4p7XNiP8r6rsx',
 '0fv2KH6hac06J86hBUTcSf',
 '4ZLzoOkj0MPWrTLvooIuaa',
 '3ee8Jmje8o58CHK66QrVC2',
 '2JvzF1RMd7lE3KmFlsyZD8',
 '0cLS7FslBJgsuWwJNRnPWB',
 '285pBltuF7vW8TeWk8hdRR',
 '2P91MQbaiQOfbiz9VqhqKQ',
 '5vdp5UmvTsnMEMESIF2Ym7',
 '45ZGEdLxCZvdMX9jWlsAAx',
 '2RSHsoi04658QL5xgQVov3',
 '1nXZnTALNXiPlvXotqHm66',
 '2xLMifQCjDGFmkHkpNLD9h',
 '7K7MUBCnzgBAvMVW2RTWNs',
 '1Ej96GIBCTvgH7tNX1r3qr',
 '7uH27oIt4a6cIFCA8ZPcyG',
 '1zB4vmk8tFRmM9UULNzbLB',
 '6zeeWid2sgw4lap2jV61PZ',
 '7KbF6AdprOXEEHlsq11Z6d',
 '6v0lAdFF4haL8xjBIUjtOw',
 '5Gu0PDLN4YJeW75PpBSg9p',
 '2rbDhOo9Fh61Bbu23T2qCk',
 '37eGbhE1xVFSvcKkqGb6i1',
 '76cy1WJvNGJTj78UqeA5zr',
 '5MvLsT03RBp67RLoeISCdl',
 '58q2HKrzhC3ozto2nDdN4z',
 '5T8EDUDqKcs6OSOwEsfqG7',
 '3Tc57t9l2O8FwQZtQOvPXK',
 '7ef4DlsgrMEH11cDZd32M6',
 '63cd4JkwGgYJrbOizbfmsp',
 '4QIo4oxwzzafcBWkKjDpXY',
 '4lzxJ4jCuFDXXGkE1LmpKR',
 '2qG5sZ7Si6sdK74qLxedYM',
 '11o2y87vqODipLK98a1v34',
 '2bPGTMB5sFfFYQ2YvSmup0',
 '6fxVffaTuwjgEk5h9QyRjy',
 '15xWRPHQMeqgdkGzInx3PY',
 '0JP9xo3adEtGSdUEISiszL',
 '6TNK1SFt77MvTFoiTloiFQ',
 '5W83ErFkO3aKAIS1WMi6u0',
 '7mWFF4gPADjTQjC97CgFVt',
 '09IStsImFySgyp0pIQdqAc',
 '7g8YaUQABMal0zWe7a2ijz',
 '4Of7rzpRpV1mWRbhp5rAqG',
 '5uCax9HTNlzGybIStD3vDh',
 '0wEC0GNF74lAqrEFZJMl1m',
 '2tOOcDDYkh0PSjr6GwPEAJ',
 '3KLygy1YWhmrQx0GRs7IOx',
 '27rdGxbavYJeBphck5MZAF',
 '0YammaEkYSeo9vQYZ1OwS6',
 '3d3k8g4GTVx9EFIFlOZOEX',
 '3V8UKqhEK5zBkBb6d6ub8i',
 '7j43FohbLVulScL7S9sQZk',
 '2093eVNfimjmNjU1gk1ES7',
 '7BuHdm4lQqcyG3qTwdkxus',
 '2iFgHPoa7FNHwgLnjXzu7F',
 '1hz7SRTGUNAtIQ46qiNv2p',
 '4au7FRcgZzChASRYu279xH',
 '15f16lrsDzFeNpHYBTzHLI',
 '5vZ4IeUenK2cHub2d7yfWk',
 '5KbBGUpIZd6gVFUU8tqYiQ',
 '6FyRXC8tJUh863JCkyWqtk',
 '5lwVd35Zg6DJ4qibVMq99H',
 '278kSqsZIiYp8p3QjYAq

In [9]:
from pyspark.ml.feature import IDF

idf = IDF(inputCol="features", outputCol="tfidf").fit(cv_result)
tfidf = idf.transform(cv_result)
tfidf.show()

+-------+----------+--------------------+--------------------+--------------------+--------------------+
country| day| songs| songs_array| features| tfidf|
+-------+----------+--------------------+--------------------+--------------------+--------------------+
 hk|2019-05-23|[[st -> 141671, i...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 id|2019-05-23|[[st -> 1892259, ...|[1iWHabUgUuuKLBa0...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 be|2019-05-23|[[st -> 381505, i...|[6u7jPi22kF8CTQ3r...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 it|2019-05-23|[[st -> 2889826, ...|[45zJRuzK9e6AO527...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 vn|2019-05-23|[[st -> 105559, i...|[5KawlOMHjWeUjQtn...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 pt|2019-05-23|[[st -> 259153, i...|[1ai9FDnFQDv18WAt...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 jp|2019-05-23|[[st -> 459664, i...|[2cKiHG16JBiqHdi7...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 za|2019-05-23|[[st -> 65710, id...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 is|2019-05-23|[[st -> 94264, id...|[4szQkW1CDHqbRZGl...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 ec|2019-05-23|[[st -> 492129, i...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 lu|2019-05-23|[[st -> 23009, id...|[2kkcJ8tSDV42LdWc...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 mx|2019-05-23|[[st -> 5201633, ...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 do|2019-05-23|[[st -> 168992, i...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 es|2019-05-23|[[st -> 3286813, ...|[1c0hsvHLELX6y8qy...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 ee|2019-05-23|[[st -> 60214, id...|[1Xi84slp6FryDSCb...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 pe|2019-05-23|[[st -> 1233597, ...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,3,4,5,...|(3854,[0,1,3,4,5,...|
 nz|2019-05-23|[[st -> 472495, i...|[3HVWdVOQ0ZA45FuZ...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 se|2019-05-23|[[st -> 2086130, ...|[2kju4JyRnNmiwQYh...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 ni|2019-05-23|[[st -> 43210, id...|[1Ej96GIBCTvgH7tN...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
 ro|2019-05-23|[[st -> 65089, id...|[6u7jPi22kF8CTQ3r...|(3854,[0,1,2,3,4,...|(3854,[0,1,2,3,4,...|
+-------+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [10]:
@udf(T.MapType(T.StringType(), T.DoubleType()))
def map_weights(tfidf_vector, vocab):
  vocab = vocab.split(",")
  values = tfidf_vector.values
  indices = tfidf_vector.indices
  map_idf = {vocab[i]: float(v) for i, v in zip(indices, values)} # {6: 0.9162907318741551, 7: 0.9162907318741551, 8: 0.9162907318741551, 11: 0.9162907318741551}
  return map_idf

tfidf = tfidf.withColumn("weights", map_weights(tfidf.tfidf, F.lit(",".join(cv.vocabulary)))) #Dirty making the join array is not literal ¯\_(ツ)_/¯
tfidf.select("country", "day", "weights").show()

+-------+----------+--------------------+
country| day| weights|
+-------+----------+--------------------+
 hk|2019-05-23|[2yJuzhRAp75De9QF...|
 id|2019-05-23|[47Bg6IrMed1GPbxR...|
 be|2019-05-23|[3iXhLi33EpUfcOwa...|
 it|2019-05-23|[5Vwqy3CS2aCB6jcg...|
 vn|2019-05-23|[52iLDrSoRtf9lcgF...|
 pt|2019-05-23|[28QdaPf3o55CDIqR...|
 jp|2019-05-23|[0bdYJCO9clK9ew61...|
 za|2019-05-23|[4Y0TtDriFlN3XZoQ...|
 is|2019-05-23|[4y3OI86AEP6PQoDE...|
 ec|2019-05-23|[4cuGZb0iFyHtwaSK...|
 lu|2019-05-23|[1L8LtC2s161XwY6Y...|
 mx|2019-05-23|[6kbzFdX9KkJpAADA...|
 do|2019-05-23|[4cuGZb0iFyHtwaSK...|
 es|2019-05-23|[4cuGZb0iFyHtwaSK...|
 ee|2019-05-23|[4Y0TtDriFlN3XZoQ...|
 pe|2019-05-23|[4cuGZb0iFyHtwaSK...|
 nz|2019-05-23|[5zxwumO0laRosoeO...|
 se|2019-05-23|[4y3OI86AEP6PQoDE...|
 ni|2019-05-23|[4cuGZb0iFyHtwaSK...|
 ro|2019-05-23|[4y3OI86AEP6PQoDE...|
+-------+----------+--------------------+
only showing top 20 rows

In [11]:
eDF = tfidf.select(F.explode(tfidf.weights).alias("id", "weight"), "country", "day")
eDF.show()

+--------------------+-------------------+-------+----------+
 id| weight|country| day|
+--------------------+-------------------+-------+----------+
2yJuzhRAp75De9QFc...| 3.449987545831587| hk|2019-05-23|
4YGSz1G203D5bD3Cj...| 3.449987545831587| hk|2019-05-23|
1eId3TVzGrwv73YAW...| 3.449987545831587| hk|2019-05-23|
4Y0TtDriFlN3XZoQx...| 1.7452394535931621| hk|2019-05-23|
4y3OI86AEP6PQoDE6...|0.03226086221822148| hk|2019-05-23|
1epcsuCSYlJW0PU6X...| 3.449987545831587| hk|2019-05-23|
53CJANUxooaqGOtds...|0.11778303565638346| hk|2019-05-23|
5KawlOMHjWeUjQtnu...|0.11778303565638346| hk|2019-05-23|
6LsAAHotRLMOHfCsS...|0.04879016416943205| hk|2019-05-23|
7BKLCZ1jbUBVqRi2F...| 1.252762968495368| hk|2019-05-23|
0uUNN1nSoUx1A4fkD...| 0.4542552722775964| hk|2019-05-23|
5icOoE6VgqFKohjWW...|0.38193461069797024| hk|2019-05-23|
7qiZfU4dY1lWllzX7...|0.17284281283941083| hk|2019-05-23|
47RgWWTJno746wCIn...| 3.449987545831587| hk|2019-05-23|
5hVghJ4KaYES3BFUA...| 0.3364722366212129| hk|2019-05-23|
1zAFaIAd1kek5IEi5...| 3.449987545831587| hk|2019-05-23|
09IStsImFySgyp0pI...| 1.0986122886681098| hk|2019-05-23|
0PG9fbaaHFHfre2gU...| 1.9459101490553132| hk|2019-05-23|
0tgVpDi06FyKpA1z0...| 0.5055485666651469| hk|2019-05-23|
6QcERmfQKHKtRE4yW...| 2.7568403652716422| hk|2019-05-23|
+--------------------+-------------------+-------+----------+
only showing top 20 rows

In [12]:
eDF.count()

Out[61]: 12400

In [13]:
audio_df = spark.table("song_features_table")
audio_df.show()
audio_df.printSchema()

+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
acousticness|danceability|duration_ms|energy|instrumentalness|key|liveness|loudness|mode| s_id|speechiness| tempo|time_signature|valence|
+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
 0.841| 0.304| 173321| 0.28| 0.0| 2| 0.0825| -9.66| 0|77kmchibFTQvcwQJC...| 0.0334|116.725| 4| 0.108|
 0.257| 0.751| 162528| 0.756| 0.0| 9| 0.0582| -5.453| 0|5tRa5hnltiyqY63rW...| 0.0375|129.954| 4| 0.644|
 0.127| 0.675| 299960| 0.771| 6.16E-6| 8| 0.0935| -5.062| 1|0sOpGWXWkxNDev51D...| 0.0282|132.038| 4| 0.619|
 0.0351| 0.477| 204984| 0.755| 0.0| 10| 0.1| -5.781| 0|1ejlLAA420azVRdgY...| 0.163| 81.283| 4| 0.586|
 0.0225| 0.665| 176121| 0.631| 0.0| 0| 0.305| -6.365| 1|7ad3XBdqiGoGIuJ83...| 0.0402|141.093| 4| 0.581|
 0.412| 0.765| 185654| 0.751| 0.0| 1| 0.169| -4.884| 0|2NpuceKgvkdmHG3vJ...| 0.236|148.028| 4| 0.61|
 0.589| 0.868| 236388| 0.751| 0.0| 7| 0.112| -5.426| 1|7LQtFOR1rnk9T8cKt...| 0.0738|127.973| 4| 0.785|
 0.072| 0.724| 177600| 0.646| 1.57E-4| 10| 0.0923| -6.475| 1|3GSMdtJphymHEsR8K...| 0.0352|112.019| 4| 0.957|
 0.0539| 0.671| 159209| 0.748| 1.8E-6| 9| 0.102| -6.47| 0|4rqMsKk1nEjL275UH...| 0.29| 96.29| 4| 0.192|
 0.112| 0.585| 247466| 0.735| 0.0| 10| 0.301| -5.206| 1|3fhvRxPQ1r7DP0JVQ...| 0.0479|111.058| 4| 0.405|
 0.539| 0.785| 189414| 0.701| 1.09E-4| 10| 0.854| -5.702| 1|4RdYKxFtC8j35NEi2...| 0.038|103.004| 4| 0.498|
 0.294| 0.655| 158907| 0.806| 0.0| 4| 0.113| -5.317| 0|56QinE0AmuVHDQGqb...| 0.219| 96.797| 4| 0.752|
 0.224| 0.678| 186680| 0.739| 0.0| 1| 0.0625| -4.479| 1|0qqUNsGRZVT9dlru6...| 0.0733|118.016| 4| 0.575|
 0.42| 0.663| 210000| 0.69| 0.0| 11| 0.11| -4.871| 0|0d4PvVcC2GxJoVm76...| 0.266|129.832| 4| 0.715|
 0.68| 0.831| 220328| 0.741| 1.04E-6| 0| 0.104| -5.866| 1|7nY4dQPdqg5apceI2...| 0.043|121.995| 4| 0.787|
 0.398| 0.747| 169090| 0.505| 3.05E-5| 4| 0.0868| -11.173| 0|4HjRpvW35frIlT33q...| 0.366|148.107| 4| 0.757|
 0.093| 0.847| 165504| 0.608| 0.0| 11| 0.132| -7.298| 1|0GrklkkWttHKbQ8XX...| 0.195|100.005| 4| 0.668|
 0.0392| 0.675| 201240| 0.662| 0.0| 1| 0.128| -4.989| 1|73NbHgUZ51H65Bfo2...| 0.0318|134.936| 4| 0.536|
 0.00242| 0.746| 207660| 0.699| 0.0| 1| 0.363| -9.243| 1|76gf6DpUXDvgXnGsI...| 0.0705| 141.02| 4| 0.466|
 0.226| 0.692| 247164| 0.86| 0.0| 7| 0.119| -4.257| 0|5xx3UTSo9aevmdGI8...| 0.437|133.885| 4| 0.566|
+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
only showing top 20 rows

root
-- acousticness: double (nullable = true)
-- danceability: double (nullable = true)
-- duration_ms: long (nullable = true)
-- energy: double (nullable = true)
-- instrumentalness: double (nullable = true)
-- key: long (nullable = true)
-- liveness: double (nullable = true)
-- loudness: double (nullable = true)
-- mode: long (nullable = true)
-- s_id: string (nullable = true)
-- speechiness: double (nullable = true)
-- tempo: double (nullable = true)
-- time_signature: long (nullable = true)
-- valence: double (nullable = true)

In [14]:
audio_df = audio_df.select("s_id","acousticness","danceability","energy","liveness","loudness","speechiness","tempo","valence")

In [15]:
eDF = eDF.join(audio_df, eDF.id == audio_df.s_id,how='left')
eDF.show()

+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
 id| weight|country| day| s_id|acousticness|danceability|energy|liveness|loudness|speechiness| tempo|valence|
+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
2yJuzhRAp75De9QFc...| 3.449987545831587| hk|2019-05-23|2yJuzhRAp75De9QFc...| 0.485| 0.502| 0.583| 0.124| -7.327| 0.0343|135.931| 0.215|
4YGSz1G203D5bD3Cj...| 3.449987545831587| hk|2019-05-23|4YGSz1G203D5bD3Cj...| 0.706| 0.728| 0.336| 0.194| -14.349| 0.12|124.033| 0.418|
1eId3TVzGrwv73YAW...| 3.449987545831587| hk|2019-05-23|1eId3TVzGrwv73YAW...| 0.834| 0.65| 0.425| 0.12| -7.816| 0.0266| 75.988| 0.403|
4Y0TtDriFlN3XZoQx...| 1.7452394535931621| hk|2019-05-23|4Y0TtDriFlN3XZoQx...| 0.0875| 0.641| 0.537| 0.115| -5.452| 0.0282|112.864| 0.0681|
4y3OI86AEP6PQoDE6...|0.03226086221822148| hk|2019-05-23|4y3OI86AEP6PQoDE6...| 0.0427| 0.842| 0.734| 0.106| -5.065| 0.0588|137.958| 0.952|
1epcsuCSYlJW0PU6X...| 3.449987545831587| hk|2019-05-23|1epcsuCSYlJW0PU6X...| 0.257| 0.493| 0.681| 0.195| -5.613| 0.0322|135.957| 0.22|
53CJANUxooaqGOtds...|0.11778303565638346| hk|2019-05-23|53CJANUxooaqGOtds...| 0.0361| 0.908| 0.517| 0.0949| -6.063| 0.129|136.019| 0.471|
5KawlOMHjWeUjQtnu...|0.11778303565638346| hk|2019-05-23|5KawlOMHjWeUjQtnu...| 0.0923| 0.645| 0.862| 0.192| -4.757| 0.0965|119.991| 0.798|
6LsAAHotRLMOHfCsS...|0.04879016416943205| hk|2019-05-23|6LsAAHotRLMOHfCsS...| 0.524| 0.692| 0.809| 0.147| -4.198| 0.0602|123.911| 0.864|
7BKLCZ1jbUBVqRi2F...| 1.252762968495368| hk|2019-05-23|7BKLCZ1jbUBVqRi2F...| 0.414| 0.748| 0.524| 0.111| -5.599| 0.0338| 95.01| 0.661|
0uUNN1nSoUx1A4fkD...| 0.4542552722775964| hk|2019-05-23|0uUNN1nSoUx1A4fkD...| 0.0737| 0.682| 0.656| 0.353| -4.67| 0.0435|130.089| 0.607|
5icOoE6VgqFKohjWW...|0.38193461069797024| hk|2019-05-23|5icOoE6VgqFKohjWW...| 0.0623| 0.791| 0.565| 0.156| -3.933| 0.0439| 99.961| 0.181|
7qiZfU4dY1lWllzX7...|0.17284281283941083| hk|2019-05-23|7qiZfU4dY1lWllzX7...| 0.581| 0.825| 0.652| 0.0931| -3.183| 0.0802| 95.977| 0.931|
47RgWWTJno746wCIn...| 3.449987545831587| hk|2019-05-23|47RgWWTJno746wCIn...| 0.67| 0.495| 0.503| 0.14| -6.838| 0.032|185.088| 0.333|
5hVghJ4KaYES3BFUA...| 0.3364722366212129| hk|2019-05-23|5hVghJ4KaYES3BFUA...| 0.23| 0.554| 0.498| 0.795| -8.866| 0.0685| 79.635| 0.413|
1zAFaIAd1kek5IEi5...| 3.449987545831587| hk|2019-05-23|1zAFaIAd1kek5IEi5...| 0.456| 0.639| 0.659| 0.235| -5.356| 0.0307|111.024| 0.54|
09IStsImFySgyp0pI...| 1.0986122886681098| hk|2019-05-23|09IStsImFySgyp0pI...| 0.171| 0.753| 0.657| 0.112| -3.061| 0.0449| 107.01| 0.437|
0PG9fbaaHFHfre2gU...| 1.9459101490553132| hk|2019-05-23|0PG9fbaaHFHfre2gU...| 0.0642| 0.747| 0.57| 0.0832| -6.711| 0.081|133.992| 0.65|
0tgVpDi06FyKpA1z0...| 0.5055485666651469| hk|2019-05-23|0tgVpDi06FyKpA1z0...| 0.163| 0.599| 0.448| 0.106| -6.312| 0.0232| 95.05| 0.168|
6QcERmfQKHKtRE4yW...| 2.7568403652716422| hk|2019-05-23|6QcERmfQKHKtRE4yW...| 0.4| 0.502| 0.458| 0.0917| -9.294| 0.0383| 75.067| 0.289|
+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
only showing top 20 rows

In [16]:
#Remove all those songs that do not have audio features
eDF = eDF.filter("s_id is not NULL")
eDF.show(10)

+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
 id| weight|country| day| s_id|acousticness|danceability|energy|liveness|loudness|speechiness| tempo|valence|
+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
2yJuzhRAp75De9QFc...| 3.449987545831587| hk|2019-05-23|2yJuzhRAp75De9QFc...| 0.485| 0.502| 0.583| 0.124| -7.327| 0.0343|135.931| 0.215|
4YGSz1G203D5bD3Cj...| 3.449987545831587| hk|2019-05-23|4YGSz1G203D5bD3Cj...| 0.706| 0.728| 0.336| 0.194| -14.349| 0.12|124.033| 0.418|
1eId3TVzGrwv73YAW...| 3.449987545831587| hk|2019-05-23|1eId3TVzGrwv73YAW...| 0.834| 0.65| 0.425| 0.12| -7.816| 0.0266| 75.988| 0.403|
4Y0TtDriFlN3XZoQx...| 1.7452394535931621| hk|2019-05-23|4Y0TtDriFlN3XZoQx...| 0.0875| 0.641| 0.537| 0.115| -5.452| 0.0282|112.864| 0.0681|
4y3OI86AEP6PQoDE6...|0.03226086221822148| hk|2019-05-23|4y3OI86AEP6PQoDE6...| 0.0427| 0.842| 0.734| 0.106| -5.065| 0.0588|137.958| 0.952|
1epcsuCSYlJW0PU6X...| 3.449987545831587| hk|2019-05-23|1epcsuCSYlJW0PU6X...| 0.257| 0.493| 0.681| 0.195| -5.613| 0.0322|135.957| 0.22|
53CJANUxooaqGOtds...|0.11778303565638346| hk|2019-05-23|53CJANUxooaqGOtds...| 0.0361| 0.908| 0.517| 0.0949| -6.063| 0.129|136.019| 0.471|
5KawlOMHjWeUjQtnu...|0.11778303565638346| hk|2019-05-23|5KawlOMHjWeUjQtnu...| 0.0923| 0.645| 0.862| 0.192| -4.757| 0.0965|119.991| 0.798|
6LsAAHotRLMOHfCsS...|0.04879016416943205| hk|2019-05-23|6LsAAHotRLMOHfCsS...| 0.524| 0.692| 0.809| 0.147| -4.198| 0.0602|123.911| 0.864|
7BKLCZ1jbUBVqRi2F...| 1.252762968495368| hk|2019-05-23|7BKLCZ1jbUBVqRi2F...| 0.414| 0.748| 0.524| 0.111| -5.599| 0.0338| 95.01| 0.661|
+--------------------+-------------------+-------+----------+--------------------+------------+------------+------+--------+--------+-----------+-------+-------+
only showing top 10 rows

In [17]:
@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def weighted_average(audio_feaures, weight):
  #Compute weighted average
  return sum(audio_feaures * weight) / sum(weight)

weighted_df = eDF.groupBy("country").agg(weighted_average(eDF["acousticness"], eDF["weight"]).alias("acousticness"),
                           weighted_average(eDF["danceability"], eDF["weight"]).alias("danceability"),
                           weighted_average(eDF["energy"], eDF["weight"]).alias("energy"),
                           weighted_average(eDF["liveness"], eDF["weight"]).alias("liveness"),
                           weighted_average(eDF["loudness"], eDF["weight"]).alias("loudness"),
                           weighted_average(eDF["speechiness"], eDF["weight"]).alias("speechiness"),
                           weighted_average(eDF["tempo"], eDF["weight"]).alias("tempo"),
                           weighted_average(eDF["valence"], eDF["weight"]).alias("valence"))
weighted_df.show(62, truncate=False)

+-------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+------------------+-------------------+
country|acousticness |danceability |energy |liveness |loudness |speechiness |tempo |valence |
+-------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+------------------+-------------------+
cr |0.2819460170183172 |0.7070841778113713|0.660984856765501 |0.17405737595644688|-5.37926525015146 |0.11166033799790946 |122.15217925362356|0.6182497988109071 |
us |0.2286178251251046 |0.7079947515127591|0.5974201556112845 |0.16100466181635284|-6.678468701201627 |0.16212583492342386 |119.94524940907564|0.45477113485759457|
cl |0.26790917385831814|0.7471208236615059|0.6877151230410453 |0.16044587796578108|-5.299305223293402 |0.12740864694664253 |123.76487328862636|0.6114341247249203 |
il |0.35467218093100994|0.6409355487288954|0.5992282595743953 |0.1685691317491257 |-6.535853352115761 |0.07495934678524303 |119.94062106184153|0.4861304797324885 |
ro |0.24542114517600705|0.6999878933977529|0.6431004654864009 |0.17863777327382538|-6.674458871842596 |0.16831947376993714 |123.11555864917592|0.4680846471745614 |
jp |0.1831564940204685 |0.5666595098725636|0.74285969295776 |0.18601259669724052|-4.572395411671807 |0.057302292467739775|121.82070413492498|0.5443875665481518 |
lv |0.22709279004506736|0.68748624770051 |0.6499531204821304 |0.1736245320459713 |-6.990648068906801 |0.13119353987077023 |122.93099355822862|0.4850878115481176 |
vn |0.40469322903006794|0.6277968651658147|0.5711447245102017 |0.16882638954772766|-6.581571565104112 |0.07548647964339487 |119.83176205472361|0.4017220192620458 |
pl |0.2731612076286827 |0.7021045096475634|0.676170356493615 |0.1699263683321479 |-7.073573537975003 |0.16480333246888093 |124.29757974133729|0.4794799756069797 |
za |0.2427424159633572 |0.649137928252275 |0.6356332113196457 |0.1775546805991936 |-6.66709980056646 |0.11856360308325456 |118.1392999028424 |0.41309505397001933|
sk |0.22846238069238406|0.6691975811268361|0.6663171557392029 |0.17318419304740132|-6.8138553931139505|0.15426157898445722 |120.37887950464865|0.40997674801681444|
pt |0.3175400700194095 |0.6936504795024495|0.6198071104845291 |0.1624987685902225 |-7.1500572600832255|0.1565854602520122 |119.82163877936055|0.45945797108590525|
in |0.30112442975229714|0.6567689873018766|0.6697367424594505 |0.16875843600522292|-6.039285275343181 |0.09015678321291604 |114.28498549071263|0.5183637856974549 |
au |0.27018243569342737|0.6748216146675766|0.6061418904460755 |0.1694091662847214 |-7.016556800644174 |0.09895073121385242 |118.71619087291532|0.45861565556291 |
ni |0.2706390336654332 |0.7181983471819051|0.676905694700808 |0.1560496805296503 |-5.169919761235662 |0.11102898732073911 |122.59407256112618|0.6062781026678412 |
sg |0.3519840705875763 |0.5834854674471491|0.5755918840905513 |0.15633866666275018|-6.554444049619882 |0.05790967171455036 |124.53423344479052|0.408105055451685 |
hk |0.441057862346232 |0.5531530241419675|0.5415349629087975 |0.15827139830422987|-7.0837548000184185|0.04960737028873384 |127.04963384901218|0.38944758070503177|
be |0.24471781753983932|0.671268599534729 |0.6618818739960868 |0.17909726488418376|-6.58546378819963 |0.1140959167291111 |116.66660230766064|0.4597699020059042 |
tw |0.4172016199304557 |0.5592140487987393|0.5437225828185339 |0.17345014350321547|-7.179281387749338 |0.05855637116377242 |123.25161054430093|0.36848048781347087|
gb |0.2581859310040629 |0.676332687607854 |0.6434221052984607 |0.1671392835781916 |-6.685417592301803 |0.14106349036353477 |121.09667886701531|0.4966855614690971 |
ec |0.2824732335850619 |0.7248072300325565|0.6876759577616355 |0.1563753715274317 |-5.193143109670877 |0.11649736024891663 |124.21753542722816|0.6459152692535176 |
co |0.2547098015661655 |0.7332472620797647|0.6964254062664667 |0.1745381993253586 |-4.99786887707138 |0.11312934408911175 |121.233918002708

In [18]:
#Maybe scale the data
#Apply K-means